In [1]:
import torch
from torch import nn
import numpy as np
from torch.autograd import Variable
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import Sampler
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, PackedSequence
import cPickle as pickle
import pandas as pd
from random import shuffle
from torch import optim
from torch.nn import functional as F 
from tqdm import tqdm,tqdm_notebook
from torch.nn.parameter import Parameter

from tensorboardX import SummaryWriter


use_cuda = torch.cuda.is_available()
torch.cuda.device(0)

In [2]:
names = ["airplane","bench","bowl","cone","desk","flower_pot",
         "keyboard","mantel","person","radio","sofa","table",
         "tv_stand","xbox","bathtub","bookshelf","car","cup",
         "door","glass_box","lamp","monitor","piano","range_hood",
         "stairs","tent","vase","bed","bottle","chair","curtain",
         "dresser","guitar","laptop","night_stand","plant",
         "sink","stool","toilet","wardrobe"]

In [3]:
def fix_data(root_directory):
    fixed = []
    df = pd.read_pickle("singles.pkl")
    for i in range(len(df)):
        fname = df.iloc[i]['name']
        ftype = "_".join(fname.split("/")[-1].split("_")[:-1])
        fno = fname.split("_")[-1][:-4]
        fname = "{}{}_{}_0.pkl".format(root_directory,ftype,fno)
        try:
            faces = pickle.load(open(fname,'rb'))
            entry = [fname,ftype,len(faces)]
            fixed.append(entry)
        except IOError:
            continue
    return fixed
#f = fix_data("../CleanedModels/pickle/")
#df = pd.DataFrame(f,columns=["name","cls_name","sizes"])
#df.to_pickle("singles.fixed.pkl")

# Data

In [4]:
class MeshSampler(Sampler):
    def __init__(self,dataset,batch_size=32):
        self.epoch_size = len(dataset)
        self.batch_size = batch_size
        self.sample_ind = range(len(dataset))
    def __len__(self):
        return (self.epoch_size + self.batch_size - 1) // self.batch_size
    def __iter__(self):
        order = range(int(self.epoch_size/self.batch_size))
        shuffle(order)
        for i in order:
            start = i * self.batch_size
            end = min(start + self.batch_size, self.epoch_size)
            yield self.sample_ind[start:end]
from numpy import cross, eye, dot
from scipy.linalg import expm, norm

def renorm(data):
    print data.shape
    med = (np.max(data,axis=0) + np.min(data,axis=0))/2
    data -= med
    print med,np.max(np.linalg.norm(data,axis=1))
    data /= np.max(np.linalg.norm(data,axis=1))
    return data

def M(axis, theta):
    return expm(cross(eye(3), axis/norm(axis)*theta))
def augment(faces,theta,rotate):
    theta = theta*30*np.pi/180.
    if rotate and theta > 0:
        R = M([0,0,1],theta)
        faces = np.dot(faces,R)
    return faces
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res
class MeshData(Dataset):
    def __init__(self,root_directory, class_we_want,max_vertices=5000):
        print "loading..."
        self.root = root_directory
        self.df = pd.read_pickle("sizes.pkl")
        print "done"
        self.df = self.df[(self.df.sizev<max_vertices) &
                          (self.df.cls_name==class_we_want) & 
                            (self.df.fname.str.contains("_0_"))]#.sort_values("sizef",ascending=False)
        self.max_vertexes = np.max(self.df.sizev.values)
    def __len__(self):
        return len(self.df)*12
    def __getitem__(self, idx):
        i = idx//12
        theta = idx%12
        fname = self.df.iloc[i]['fname']
        end = fname.split("/")[-1]
        ftype = "_".join(fname.split("/")[-1].split("_")[:-4])
        fno = fname.split("_")[-1][:-4]
        #fname = "{}{}_{}_0.pkl".format(self.root,ftype,fno)
        data = pickle.load(open(fname,'rb'))
        vertices = data['vertices']
        vertices = augment(vertices,theta,True)
        assert len(vertices) <= self.max_vertexes
        return torch.from_numpy(vertices).float()
trainer = MeshData("../CleanedModels/pickle/","chair",300)

def fill_with_zeros(x,l):
    new_x = torch.zeros(l)
    new_x[:x.size(0)]=x
    return new_x
    
def pad_packer(x):
    faces,cls_idx = zip(*x)
    cls_idx = torch.cat(cls_idx)
    max_size = faces[0].size()
    lengths = [len(i) for i in faces]
    faces = [fill_with_zeros(i,max_size) for i in faces]
    faces = Variable(torch.stack(faces,0))
    return faces,cls_idx,lengths
train_loader = DataLoader(trainer,shuffle=True)

loading...
done


# Model

In [5]:
def loss_function(output, faces, mu, logvar):
    recon_x = output
    x = faces
    CD = ChamferDistance(recon_x,x)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)
    return CD,KLD

def pairwise_dist(x, y):
    xx, yy, zz = torch.mm(x,x.t()), torch.mm(y,y.t()), torch.mm(x, y.t())
    rx = (xx.diag().unsqueeze(0).expand_as(xx))
    ry = (yy.diag().unsqueeze(0).expand_as(yy))
    P = (rx.t() + ry - 2*zz)
    return P


def ChamferDistance(x, y, dim=0):
    x,y = x.squeeze(),y.squeeze()
    #print x.size(),y.size()
    dist = pairwise_dist(x, y)
    #i,j = np.random.randint(2000),np.random.randint(2000)
    #print torch.sum(torch.pow(x[i]-y[j],2)),dist[i,j]
    xvalues, _ = dist.min(dim=dim)
    yvalues, _ = dist.min(dim=1)
    return xvalues.mean()+yvalues.mean()

#print ChamferDistance(a.cuda(),b.cuda())

In [6]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('LSTM') != -1:
        nn.init.xavier_normal(m.weight_ih_l0.data)
        nn.init.xavier_normal(m.weight_hh_l0.data)
        nn.init.constant(m.bias_ih_l0.data,0.001)
        nn.init.constant(m.bias_hh_l0.data,0.001)
    elif classname.find('Linear') != -1:
        nn.init.xavier_normal(m.weight.data)
        nn.init.constant(m.bias.data,0.001)
class VertexEncoder(nn.Module):
    def __init__(self,lstm_units,z_dim,in_dims=3,bi=True):
        super(VertexEncoder, self).__init__()
        self.hidden_size=lstm_units
        self.bi = bi
        if bi:
            self.hidden_size *= 2
        self.lstm = nn.LSTM(in_dims,lstm_units,1,batch_first=True,bidirectional=bi)
        self.mu = nn.Linear(self.hidden_size,z_dim)
        self.logvar = nn.Linear(self.hidden_size,z_dim)
    def forward(self,x,h):
        x,(h,c) = self.lstm(x,h)
        h = F.relu(h.squeeze()).view(self.hidden_size)
        mu = self.mu(h)
        logvar = self.logvar(h)
        return mu,logvar
    def initHidden(self):
        firstdim = 1
        if self.bi:
            firstdim = 2
        h0 = Variable(torch.zeros(firstdim, 1, self.hidden_size/firstdim))
        c0 = Variable(torch.zeros(firstdim, 1, self.hidden_size/firstdim))
        if use_cuda:
            return h0.cuda(),c0.cuda()
        else:
            return h0,c0
class VertexDecoder(nn.Module):
    def __init__(self,lstm_units,z_dim,in_dims=3):
        super(VertexDecoder, self).__init__()
        self.hidden_size=lstm_units
        self.z_dim = z_dim
        self.lstm = nn.LSTM(self.z_dim+in_dims,self.hidden_size)
        self.fc1 = nn.Linear(self.hidden_size,in_dims)
        self.fc_hc = nn.Linear(self.z_dim, 2*self.hidden_size)
    def forward(self,inputs,z,seq_len,hidden_cell=None):
        if hidden_cell is None:
            h = F.tanh(self.fc_hc(z))
            hidden,cell = torch.split(h,self.hidden_size,1)
            hidden_cell = (hidden.unsqueeze(0).contiguous(), cell.unsqueeze(0).contiguous())
        outputs,(hidden,cell) = self.lstm(inputs, hidden_cell)
        outputs = self.fc1(outputs)
        return outputs, (hidden,cell)

class VertexVAE(nn.Module):
    def __init__(self,z_dim=64):
        super(VertexVAE, self).__init__()
        self.encoder = VertexEncoder(lstm_units=200,z_dim=z_dim)
        self.decoder = VertexDecoder(lstm_units=400,z_dim=z_dim)
    def forward(self,x,h):
        seq_len = x.size(1)
        mu,logvar = self.encoder(x,h)
        z = self.reparameterize(mu,logvar)
        z_stack = torch.stack([z]*(seq_len)).unsqueeze(0)
        z = z.unsqueeze(0)
        inputs = torch.cat([x, z_stack],2).t()
        x,_ = self.decoder(inputs,z,seq_len)
        return x, mu, logvar, z
    def reparameterize(self, mu, sigma):
        if self.training:
            std = sigma.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            return eps.mul(std).add_(mu)
        else:
            return mu


In [7]:
vvae = VertexVAE()
vvae.apply(weights_init)
if use_cuda:
    vvae = vvae.cuda()     
optimizer = optim.Adam(vvae.parameters(),lr=0.001)

In [8]:
# tensorboard things
writer = SummaryWriter()
global_counter = 0 

#vandf.load_state_dict(torch.load("facer.tar"))

In [9]:
def log_tensors(old_tensors):
    tensors_existing = []
    for obj in gc.get_objects():
        if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
            if (hasattr(obj, 'data')):
                obj = obj.data
            if "cuda" in str(type(obj)):
                #print type(obj)
                tensors_existing.append(obj)
    if old_tensors is not None:
        found = 0
        for te in tensors_existing:
            foundit = False
            for o in old_tensors:
                if o.type()==te.type() and o.size()==te.size() and torch.equal(o,te):
                    found+=1
                    foundit = True
                    break
            if not foundit:
                print te.size()
        for o in old_tensors:
            foundit = False
            for te in tensors_existing:
                if o.type()==te.type() and o.size()==te.size() and torch.equal(o,te):
                    foundit = True
                    break
            if not foundit:
                print "deleted one"
            
                
        return tensors_existing,found
    return tensors_existing,0
    

In [10]:
# VAE only
printevery = 3
import gc
te,found = log_tensors(None)
print "starting tensors: {}".format(len(te))
#ms2s.cuda()
genevery = 100
import time
for i in range(30):
    for idx,(vertices) in enumerate(train_loader):
        optimizer.zero_grad()
        vertices = Variable(vertices)
        if use_cuda:
            #faces=faces.cuda()
            vertices=vertices.cuda()
        #print faces.size()
        seq_len = vertices.size(1)
        print seq_len
        writer.add_scalars("sizes",{"vlen":seq_len},global_step=global_counter)
        te,found = log_tensors(te)
        print "tensors before forward: {} {}".format(len(te),found)
        now = time.time()
        h = vvae.encoder.initHidden()
        out_vertexes,mu,logvar,z = vvae(vertices,h)
        forward_time = time.time()-now
        CD,KLD = loss_function(out_vertexes,vertices,mu,logvar)
        now = time.time()
        loss = CD + KLD
        loss.backward()
        nn.utils.clip_grad_norm(vvae.parameters(), 1.)
        optimizer.step()
        te,found = log_tensors(te)
        print "tensors after forward: {} {}".format(len(te),found)
        backward_time = time.time()-now
        writer.add_scalars("timing/master",{"forward": forward_time, "backward": backward_time},global_counter)
        #print "forward: {}, backward: {}".format(forward_time,backward_time)
        writer.add_scalars("losses",{"CD_Vertex": CD.data.cpu()[0],
                                     "KLD_Vertex": KLD.data.cpu()[0],
                                    "total":loss},global_counter)
        te,found = log_tensors(te)
        print "tensors after logging: {} {}".format(len(te),found)
        global_counter+=1

SyntaxError: can't delete () (<ipython-input-10-c26e4287484c>, line 32)

In [ ]:
torch.save(vandf.cpu().state_dict(),"facer.tar")

In [ ]:
def write_off(data,fname):
    data = data.reshape(-1,3)
    points = set()
    for d in data:
        points.add(tuple(d))
    vertexes = list(points)
    print len(vertexes),len(data)
    faces = []
    for d in data:
        faces.append(vertexes.index(tuple(d)))
    faces = np.array(faces)
    faces = faces.reshape(-1,3)
    with open(fname,'w') as openfile:
        openfile.write("OFF\n")
        openfile.write("{} {} 0\n".format(len(vertexes),len(faces)))
        for v in vertexes:
            openfile.write("{} {} {}\n".format(v[0],v[1],v[2]))
        for f in faces:
            openfile.write("3 {} {} {}\n".format(f[0],f[1],f[2]))
